In [1]:
with open('paloozabot.log','r') as file:
    logs = file.readlines() 

In [2]:
line = logs[1]

In [3]:
line

'05/01/2024 03:33:36 PM:WARNING:discord.client:client:295:__init__: PyNaCl is not installed, voice will NOT be supported\n'

In [8]:
import openai
from dotenv import load_dotenv,find_dotenv
import os

_ = load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

openai.api_key = api_key

In [9]:
def get_completion(prompt, model="gpt-4"):
    '''basic completion from gpt4'''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [10]:
get_completion("who won the world series in 1980?")

'The Philadelphia Phillies won the World Series in 1980.'

In [47]:
with open('log_extraction_prompt.md','r') as f:
    base_prompt = f.read()

In [57]:
base_prompt = base_prompt.replace("\n"," ")
test_input = base_prompt.format(log="""05/02/2024 02:36:05 PM:INFO:discord:bot:50:on_message: Message from a_na4603 in #project: [paloozabot](https://github.co
m (ID: 1233476522032103444): @everyone 

update:

we have given paloozabot robust logging capabilities. we plan to use the logs for understanding system (bot) and user be
havior. logs need to be extracted, transformed, and loaded into a database - we laid the groundwork for this here:

[paloozabot logs data model sketch](https://docs.google.com/document/d/1634WYdIV1gSJxX5kSBqkdombvkzXfMHB75_AlZfM1xs/edit
?usp=sharing), Message ID: 1235600710037540895""")
print(test_input)

## IDENTIY AND PURPOSE  you are an expert in extracting log data from a log file and formatting it as JSON.  ## STEPS  - read log data carefully - compare it to the following SQL table definitions  ```sql TABLE Message (     Message_time TIMESTAMP,     User_id VARCHAR(32),     Channel_name VARCHAR(32),     Channel_id INTEGER,     Message TEXT,     Mentions TEXT[],     Emojis TEXT[] );   TABLE Command (     Message_time TIMESTAMP,     User_id VARCHAR(32),     Command VARCHAR(32) );   TABLE System (     Datetime TIMESTAMP,     Level VARCHAR(8),     Module TEXT,     Context TEXT,     Message TEXT );  ```  - generate key:value pairs from the log that correspond to the fields in the table definitions  ## OUTPUT INSTRUCTIONS  you only output valid JSON and nothing else.  ## EXAMPLE  input:  05/01/2024 03:33:36 PM:WARNING:discord.client:client:295:__init__: PyNaCl is not installed, voice will NOT be supported\n  JSON output:  {  "Datetime": "2024-05-01T15:33:36",  "Level": "WARNING",  "Module

In [58]:
test_output = get_completion(test_input)

In [60]:
print(test_output)

{
  "Message_time": "2024-05-02T14:36:05",
  "User_id": "a_na4603",
  "Channel_name": "#project",
  "Channel_id": 1233476522032103444,
  "Message": "@everyone \n\nupdate:\n\nwe have given paloozabot robust logging capabilities. we plan to use the logs for understanding system (bot) and user behavior. logs need to be extracted, transformed, and loaded into a database - we laid the groundwork for this here:\n\n[paloozabot logs data model sketch](https://docs.google.com/document/d/1634WYdIV1gSJxX5kSBqkdombvkzXfMHB75_AlZfM1xs/edit?usp=sharing), Message ID: 1235600710037540895",
  "Mentions": ["@everyone"],
  "Emojis": []
}


In [61]:
len(test_input)/4

449.75

In [62]:
len(test_output)/4

156.5

In [65]:
(10/1_000_000)*450

0.0045000000000000005

In [ ]:
# ~4 characters = 1 token

In [51]:
import json

json.loads(test_output)

{'Datetime': '2024-05-01T15:33:36',
 'Level': 'DEBUG',
 'Module': 'discord.client',
 'Context': 'client:1989:event',
 'Message': 'on_ready has successfully been registered as an event'}

In [52]:
errors = 0
for i in range(20):

    test_output = get_completion(test_input)

    try:
        json.loads(test_output)
    except:
        errors += 1

print(errors)

0


In [39]:
# 50% error rate

# chat bots are too chatty for this use case
example_prompt = \
"""
input: {input}
JSON output: 
"""

example_prompt.format(input = "my input")

'\ninput: my input\nJSON output: \n'

In [ ]:
# start with a log
# then get a json
# we turn it into a single tabular record
# then we transfer it to the database


# the AI is giving us different jsons (3 different kinds)
# so it spits out a json
# and we need to know what table it belongs to.

In [53]:
with open('paloozabot.log','r') as file:
    logs = file.read() 

In [56]:
len(logs)/4

8041.0